# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mil'kovo,55.6078,37.7956,3.73,81,100,5.32,RU,1696885276
1,1,yar,58.2490,52.1026,2.46,97,99,4.49,RU,1696885276
2,2,champasak,14.7500,106.0000,22.18,99,81,0.71,LA,1696885276
3,3,princeville,45.7668,-61.2985,16.97,79,0,2.03,CA,1696885276
4,4,arvayheer,46.2639,102.7750,4.28,42,4,1.99,MN,1696885277


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["City"]
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Define the criteria to select ideal city
max_temperature = (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)
max_wind_speed = city_data_df["Wind Speed"] < 4.5
max_cloudiness = city_data_df["Cloudiness"] == 0
select_city = city_data_df[max_temperature & max_wind_speed & max_cloudiness]

# Drop any rows with null values
select_city = select_city.dropna()

# Display sample data
select_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,port hedland,-20.3167,118.5667,21.03,32,0,2.57,AU,1696885247
41,41,hassi messaoud,31.6804,6.0729,25.92,36,0,2.57,DZ,1696885287
74,74,tumayr,25.7039,45.8680,26.34,23,0,2.30,SA,1696885300
75,75,'ain benian,36.8028,2.9219,21.03,77,0,3.19,DZ,1696885300
97,97,dolores,-36.3132,-57.6792,24.00,51,0,3.18,AR,1696885305
132,132,rafsanjan,30.4067,55.9939,24.22,15,0,2.49,IR,1696885319
179,179,raton,36.9034,-104.4391,24.34,14,0,3.09,US,1696885332
187,187,la rinconada,37.4873,-5.9826,25.59,60,0,3.60,ES,1696885335
360,360,karratha,-20.7377,116.8463,24.42,48,0,2.31,AU,1696885376
472,472,rio grande,-32.0350,-52.0986,24.06,76,0,4.14,BR,1696885172


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = select_city[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
29,port hedland,AU,-20.3167,118.5667,32,
41,hassi messaoud,DZ,31.6804,6.0729,36,
74,tumayr,SA,25.7039,45.8680,23,
75,'ain benian,DZ,36.8028,2.9219,77,
97,dolores,AR,-36.3132,-57.6792,51,
132,rafsanjan,IR,30.4067,55.9939,15,
179,raton,US,36.9034,-104.4391,14,
187,la rinconada,ES,37.4873,-5.9826,60,
360,karratha,AU,-20.7377,116.8463,48,
472,rio grande,BR,-32.0350,-52.0986,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city"s latitude and longitude to the params dictionary
    filter = f"circle:{lng},{lat},{radius}"
    bias = f"proximity:{lng},{lat}"
    
    params = {
        "categories": "accommodation.hotel",
        "limit": 1,
        "filter":filter,
        "bias":bias,
        "apiKey":geoapify_key
    }
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port hedland - nearest hotel: The Esplanade Hotel
hassi messaoud - nearest hotel: AMC
tumayr - nearest hotel: No hotel found
'ain benian - nearest hotel: Mercure Africana
dolores - nearest hotel: Plaza Hotel
rafsanjan - nearest hotel: هتل جهانگردی
raton - nearest hotel: Travelodge
la rinconada - nearest hotel: Hotel Torre de los Guzmanes
karratha - nearest hotel: Karratha International Hotel
rio grande - nearest hotel: Hotel Vila Moura Executivo
l'isle-jourdain - nearest hotel: L'Échappée Belle
anta - nearest hotel: No hotel found
ilha de mocambique - nearest hotel: Ruby Backpackers
paso de los toros - nearest hotel: Hotel Italiana
teguise - nearest hotel: Finca Las Laderas
gulf shores - nearest hotel: Beachside Resort


,City,Country,Lat,Lng,Humidity,Hotel Name
29,port hedland,AU,-20.3167,118.5667,32,The Esplanade Hotel
41,hassi messaoud,DZ,31.6804,6.0729,36,AMC
74,tumayr,SA,25.7039,45.8680,23,No hotel found
75,'ain benian,DZ,36.8028,2.9219,77,Mercure Africana
97,dolores,AR,-36.3132,-57.6792,51,Plaza Hotel
132,rafsanjan,IR,30.4067,55.9939,15,هتل جهانگردی
179,raton,US,36.9034,-104.4391,14,Travelodge
187,la rinconada,ES,37.4873,-5.9826,60,Hotel Torre de los Guzmanes
360,karratha,AU,-20.7377,116.8463,48,Karratha International Hotel
472,rio grande,BR,-32.0350,-52.0986,76,Hotel Vila Moura Executivo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=600,
    frame_height=500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)